# Лабораторная работа № 5 по дисциплине "Системы искусственного интеллекта"

# Задание лабораторной работы
0. Выбор датасета:
    - Датасет о пассажирах Титаника: [Titanic Dataset](https://www.kaggle.com/c/titanic)
    - ~~Датасет о диабете: [Diabetes Dataset](https://www.kaggle.com/uciml/pima-indians-diabetes-database)~~
1. Загрузите выбранный датасет и выполните предварительную обработку данных. 
2. Получите и визуализируйте (графически) статистику по датасету (включая количество, среднее значение, стандартное отклонение, минимум, максимум и различные квантили).
3. Разделите данные на обучающий и тестовый наборы в соотношении, которое вы считаете подходящим.
4. Реализуйте логистическую регрессию "с нуля" без использования сторонних библиотек, кроме NumPy и Pandas. Ваша реализация логистической регрессии должна включать в себя:
    - Функцию для вычисления гипотезы (sigmoid function).
    - Функцию для вычисления функции потерь (log loss).
    - Метод обучения, который включает в себя градиентный спуск.
    - Возможность варьировать гиперпараметры, такие как коэффициент обучения (learning rate) и количество итераций.
5. Исследование гиперпараметров:
    - Проведите исследование влияния гиперпараметров на производительность модели. Варьируйте следующие гиперпараметры:
        - Коэффициент обучения (learning rate).
        - Количество итераций обучения.
        - Метод оптимизации (например, градиентный спуск или оптимизация Ньютона).
6. Оценка модели:
    - Для каждой комбинации гиперпараметров оцените производительность модели на тестовом наборе данных, используя метрики, такие как accuracy, precision, recall и F1-Score.

## Выполнение лабораторной работы
### 0. Установка и импорт библиотек, загрузка датасета

Сначала установим библиотеки, если они уже не установлены.

In [1]:
!pip install numpy pandas matplotlib


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Импортируем необходимые библиотеки.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Импортируем датасет:

In [3]:
df_train = pd.read_csv('../datasets/titanic/train.csv')
print(df_train.describe(include='all'))
df_train.head()

        PassengerId    Survived      Pclass                 Name   Sex  \
count    891.000000  891.000000  891.000000                  891   891   
unique          NaN         NaN         NaN                  891     2   
top             NaN         NaN         NaN  Dooley, Mr. Patrick  male   
freq            NaN         NaN         NaN                    1   577   
mean     446.000000    0.383838    2.308642                  NaN   NaN   
std      257.353842    0.486592    0.836071                  NaN   NaN   
min        1.000000    0.000000    1.000000                  NaN   NaN   
25%      223.500000    0.000000    2.000000                  NaN   NaN   
50%      446.000000    0.000000    3.000000                  NaN   NaN   
75%      668.500000    1.000000    3.000000                  NaN   NaN   
max      891.000000    1.000000    3.000000                  NaN   NaN   

               Age       SibSp       Parch  Ticket        Fare Cabin Embarked  
count   714.000000  891.000000 

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df_test = pd.read_csv('../datasets/titanic/test.csv')
print(df_test.describe(include='all'))
df_test.head()

        PassengerId      Pclass                      Name   Sex         Age  \
count    418.000000  418.000000                       418   418  332.000000   
unique          NaN         NaN                       418     2         NaN   
top             NaN         NaN  Peter, Master. Michael J  male         NaN   
freq            NaN         NaN                         1   266         NaN   
mean    1100.500000    2.265550                       NaN   NaN   30.272590   
std      120.810458    0.841838                       NaN   NaN   14.181209   
min      892.000000    1.000000                       NaN   NaN    0.170000   
25%      996.250000    1.000000                       NaN   NaN   21.000000   
50%     1100.500000    3.000000                       NaN   NaN   27.000000   
75%     1204.750000    3.000000                       NaN   NaN   39.000000   
max     1309.000000    3.000000                       NaN   NaN   76.000000   

             SibSp       Parch    Ticket        Far

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


### 1. Предварительная обработка данных

> Загрузите выбранный датасет и выполните предварительную обработку данных. 

#### ~~1.1. Удаление отсутствующих значений~~

Определим процент строк в тренировочном и тестовом датасете, содержащих отсутствующие значения:

In [5]:
def na_rate_dataframe(df: pd.DataFrame) -> float:
    return df.isna().any(axis='columns').sum() / df.shape[0]

def na_rate_series(s: pd.Series) -> float:
    return s.isna().sum() / s.size

print(f'Missing values in train dataset: {(na_rate_dataframe(df_train) * 100):.2f}%')
print(f'Missing values in test dataset: {(na_rate_dataframe(df_test) * 100):.2f}%')

Missing values in train dataset: 79.46%
Missing values in test dataset: 79.19%


Большое количество отсутствующих значений свидетельствует о том, что удаление всех строк с отсутствующими значениями нецелесообразно.

### 1.2. Переобозначение
Для удобства переобозначим наборы столбцов:

In [6]:
x_train = df_train.drop(columns=['Survived'])
y_train = df_train['Survived']
x_test = df_test # df_test has no column 'Survived'

### 1.3. Импутация

Применим импутацию (восстановление) данных для столбцов, в которых не более 50% пропусков: 
- Если пропущено более 50% значений столбца в тренировочном датасете, помечаем столбец на удаление.
- Для числовых признаков используем выборочное среднее.
- Для категориальных признаков используем моду.

Выделим категориальные и числовые признаки:

In [7]:
numeric_columns = x_train.select_dtypes(include='number').columns 
categorical_columns = x_train.select_dtypes(exclude='number').columns
print(f'Numeric columns: {numeric_columns}')
print(f'Categorical columns: {categorical_columns}')

Numeric columns: Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')
Categorical columns: Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')


Помечаем на удаление столбцы с большим числом пропусков:

In [8]:
_THRESHOLD = 0.5
columns_to_delete = [
    col 
    for col in x_train.columns
    if x_train[col].isna().mean() > _THRESHOLD
]
columns_to_delete

['Cabin']

Также проверим категориальные признаки с большим числом уникальных значений ($k > 10$):

In [9]:
_MAX_CARDINALITY = 10
high_cardinality_columns = [
    col
    for col in categorical_columns
    if df_train[col].nunique() > _MAX_CARDINALITY
]
columns_to_delete += high_cardinality_columns
columns_to_delete = list(set(columns_to_delete))
columns_to_delete

['Name', 'Ticket', 'Cabin']

Также удалим столбец `PassengerId`, т. к. он, скорее всего, не привнесёт полезную информацию (это просто порядковый номер пассажира в датасете).

In [10]:
columns_to_delete += ['PassengerId']
columns_to_delete = list(set(columns_to_delete))
columns_to_delete

['PassengerId', 'Name', 'Ticket', 'Cabin']

In [11]:
x_train = x_train.drop(columns=columns_to_delete)
x_test = x_test.drop(columns=columns_to_delete)

numeric_columns = [col for col in numeric_columns if col not in columns_to_delete]
categorical_columns = [col for col in categorical_columns if col not in columns_to_delete]

print(f'Train dataset columns: {x_train.columns}')
print(f'Test dataset columns: {x_test.columns}')

Train dataset columns: Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')
Test dataset columns: Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')


Заполнение числовых признаков:

In [12]:
for col in numeric_columns:
    assert x_train[col].dtype == x_test[col].dtype, f'type {x_train[col].dtype} != type {x_test[col].dtype}'

    mean = x_train[col].mean()
    
    print(f'{col}: type {x_train[col].dtype}, empty {x_train[col].isna().sum()}, mean {mean:.6f}')
    x_train[col] = x_train[col].fillna(mean)
    x_test[col] = x_test[col].fillna(mean)
    
    # Are there no unfilled values left?
    assert x_train[col].isna().sum() == 0, \
        f'{col}: {x_train[col].isna().sum()} missing values left in train'
    assert x_test[col].isna().sum() == 0, \
        f'{col}: {x_test[col].isna().sum()} missing values left in test'

Pclass: type int64, empty 0, mean 2.308642
Age: type float64, empty 177, mean 29.699118
SibSp: type int64, empty 0, mean 0.523008
Parch: type int64, empty 0, mean 0.381594
Fare: type float64, empty 0, mean 32.204208


In [13]:
for col in categorical_columns:
    assert x_train[col].dtype == x_test[col].dtype, f'type {x_train[col].dtype} != type {x_test[col].dtype}'
    
    mode = x_train[col].mode()[0]
    
    print(f'{col}: type {x_train[col].dtype}, empty {x_train[col].isna().sum()}, unique {x_train[col].nunique()}, mode {mode}')
    x_train[col] = x_train[col].fillna(mode)
    x_test[col] = x_test[col].fillna(mode)

    assert x_train[col].isna().sum() == 0, f'{col}: {x_train[col].isna().sum()} missing values left in train'
    assert x_test[col].isna().sum() == 0, f'{col}: {x_test[col].isna().sum()} missing values left in test'

Sex: type object, empty 0, unique 2, mode male
Embarked: type object, empty 2, unique 3, mode S


#### 1.4. Кодирование категориальных признаков

Нужно закодировать 2 категориальных признака: `Sex` и `Embarked`. Кодируем их числами:

In [14]:
for dataset in (x_train, x_test):
    dataset['Sex'] = dataset['Sex'].map({'male': 0, 'female': 1}).fillna(-1)
    dataset['Embarked'] = dataset['Embarked'].map({'C': 0, 'Q': 1, 'S': 2}).fillna(-1)

x_train['Sex'].unique(), x_test['Sex'].unique(), x_train['Embarked'].unique(), x_test['Embarked'].unique()

(array([0, 1]), array([0, 1]), array([2, 0, 1]), array([1, 2, 0]))

#### 1.5. Нормализация числовых признаков

Для числовых признаков выполним нормализацию по закону:
$$
x' = \cfrac{x-\bar x}{\sigma}.
$$

Значения выборочного среднего и стандартного отклонения берём из **тренировочного** датасета.

In [15]:
x_train_norm = x_train.copy()
x_test_norm = x_test.copy()

In [16]:
x_train_numeric = x_train[numeric_columns]
_x_mean = x_train_numeric.mean(axis=0)
_x_std = x_train_numeric.std(axis=0)
_x_std[_x_std == 0] = 1

In [19]:
x_train_norm[numeric_columns] = (x_train[numeric_columns] - _x_mean) / _x_std
x_test_norm[numeric_columns] = (x_test[numeric_columns] - _x_mean) / _x_std
x_train_norm.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0.826913,0,-0.592148,0.432550,-0.473408,-0.502163,2
1,-1.565228,1,0.638430,0.432550,-0.473408,0.786404,0
2,0.826913,1,-0.284503,-0.474279,-0.473408,-0.488580,2
3,-1.565228,1,0.407697,0.432550,-0.473408,0.420494,2
4,0.826913,0,0.407697,-0.474279,-0.473408,-0.486064,2
